In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from ipywidgets import interact

# from conc_obj import EEGData
from eegdata_multi import EEGData
from utils.plt import plot_psd, plot_montage
from utils.ica import plot_ica_comp

# MNE imports
import mne
from mne.io.edf import read_raw_edf
from mne.datasets import eegbci

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

import json


***Macros***

>General use macros, importing JSON files to use as the configuration files

In [ ]:
script_path = Path().resolve()
folder = (script_path / "../").resolve()

JSON_MAIN_PATH = script_path / "config/config_main.json"
JSON_CSP_PATH = script_path / "config/config_csp.json"
JSON_GRID_PATH = script_path / "config/config_grid.json"
EVENTS_PATH = script_path / "config/events.json"

with open(JSON_MAIN_PATH, "r") as f:
    config_main = json.load(f)

with open(JSON_CSP_PATH, "r") as f:
    config_csp = json.load(f)

VERBOSE = config_main['verbose'].lower() == 'true'

L_FREQ = config_main['l_freq']
H_FREQ = config_main['h_freq']

N_SUBJECTS = config_main["n_subjects"]
N_COMPONENTS = config_main["n_components"]


***Initialization of EEG object***

>***(If the files are not locally stored, it will download them to the user system automatically)***

>***Use of functions like .filter_data() also is obligatory if there is no data stored locally***

In [ ]:
eeg_obj = EEGData(config_main, config_csp, folder, verbose=VERBOSE)
# eeg_obj.save_type_data(type="events", folder_path=folder, verbose=VERBOSE)

#* Filters data and plots PSD to see differences
# eeg_obj.filter_data()
# eeg_obj.plot_psd_ba_filt(verbose=VERBOSE)

**Basic information and montage plotting in 2D & 3D**
> ***The channel names can also be printed***

In [ ]:

#* Plots different montages in 2D & 3D
# data = eeg_obj.get_raw_h()

# ch_names = data.info["ch_names"] 

# plot_montage(eeg_obj.montage, ch_names)

**ICA(Independent Component Analysys)**
> ***The number of components that ICA will try to sort out can be changed, it is advised to use values in the range [16-64]***

> ***Ocular artifacts are also removed, since they don't contribute to the muscular movement on this evaluation***

> ***The components can also be plotted and ocular artifacts, EOG, will be clearly visible***

In [ ]:

#* Computes ICA components ( If loaded locally do not use! )
# eeg_obj.decomp_ica(n_components=N_COMPONENTS, plt_show=True, verbose=VERBOSE)

#* Plot components of ICA
# plot_ica_comp(folder / config_main["path_ica_h"])

**Specify events & create Epochs**
> ***The events used along with the JSON configuration will be crucial for the ML algorimths to work properly***


In [ ]:

#* Loads cleaned data and events
data_h, data_hf = eeg_obj.get_clean()
events_h, events_hf = eeg_obj.get_events()

#* Creates epochs and frequency bands
ev_list = config_csp["ev_mlist_one"]
epochs, freq_bands = eeg_obj.crt_epochs(data_h, events_h, ev_list, "hands", verbose=VERBOSE)

print()
epochs_data = epochs.get_data()
labels = epochs.events[:, -1]
print()


**Normalize and apply PCA(Principal Component Analysis)**
> ***Faster computation, training, testing, ...***

> ***Reduces the risk of over-fitting***

> ***Improves the accuracy of classification ML algorimths***

In [ ]:
#* Normalizes data
features_norm = eeg_obj.normalize(epochs_data)

#* Reduce dimensionality (PCA)
features_pca = eeg_obj.pca(features_norm)

In [ ]:

features, csp = eeg_obj.csp(features_pca, labels, freq_bands, verbose=VERBOSE)

#* Only use plot_patters if you are not using PCA before
# csp.plot_patterns(epochs.info, ch_type="eeg", units="Patterns (AU)", size=1.5)

#* Displays the performance of CSP along classifiers through a timeline
# eeg_obj.csp_performance(epochs, labels, clf_type='svm', verbose=False)

#* Two step CSP
# features, labels = eeg_obj.two_step_csp(epochs1, epochs2, freq_bands, verbose=VERBOSE)

#* Verify any shape
print("Shape after CSP:", features.shape)

***GridSearch - Parameter selection***

>Exhaustive search over specified parameter values for an estimator.

>The default values have been tested. Performance varies from event type selection.

In [ ]:
from utils.gridsearch import grid_finder, grid_search

# grid = grid_finder(json_grid, 'svm', 'wide')
# print(grid)
# grid_search(data, labels, pipeline, grid)

In [ ]:
from utils.pipeline import crt_pipeline

pipeline = crt_pipeline(clf=True, voting='soft')

#* Trains and evaluates model
scores = eeg_obj.cross_val(features, labels, pipeline, n_splits=7)
print("Mean score:", np.mean(scores))

In [ ]:
#* Saves filtered and concatenated data for faster loading
# eeg_obj.save_type_data(type="raw", folder_path=folder)
# eeg_obj.save_type_data(type="filtered", folder_path=folder)
# eeg_obj.save_type_data(type="ica", folder_path=folder)
# eeg_obj.save_type_data(type="clean", folder_path=folder)
# eeg_obj.save_type_data(type="epochs", folder_path=folder)